In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-07-04 07:25:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-07-04 07:25:37 (51.5 MB/s) - ‘minsearch.py’ saved [4073/4073]



In [2]:
import minsearch

/workspaces/llm-zoomcamp/01-Intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json

In [38]:
import openai
from openai import OpenAI

In [60]:
import os
from dotenv import load_dotenv

In [39]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [40]:
client = OpenAI()

In [41]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [43]:
documents = []

for course_dict in docs_raw:
    for docs in course_dict['documents']:
        docs['course'] = course_dict['course']
        documents.append(docs)

In [44]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [45]:
index = minsearch.Index(
    text_fields = ['text', 'section', 'question'],
    keyword_fields = ['course']
)

In [46]:
index.fit(documents)

In [47]:
q = 'the course has already started, can I still enroll?'

In [68]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [62]:
def build_prompt(query, search_results):
    prompt_template = """
        You are course teaching assistant, Answer the QUESTION  based on CONTEXT from FAQ database.
        Use only facts from teh CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE

        QUESTION: {question}

        CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [63]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [78]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [79]:
query = "the course has already started, can I still enroll?"
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."